# Multiple knapsacks

## Intro

This is a generalization of the knapsack problem, in which the goal is to carry the maximum value in a single backpack considering a given set of items and a maximum capacity.

In [1]:
import pandas as pd
import pyomo.environ as pyo

In [2]:
#Here it is the information about items available
data_items = pd.read_excel("..\data\multi_knapsacks.xlsx", sheet_name=0, index_col=0)

#And here about the knapsacks
data_knapsacks = pd.read_excel("..\data\multi_knapsacks.xlsx", sheet_name=1, index_col=0)

In [3]:
data_items.head()

,category,weight,volume,value,available
1,eletronics,4.661295,1.002111,1.947939,1
2,eletronics,7.514717,1.146791,5.189848,1
3,beverage,8.996028,5.010643,4.431570,1
4,clothing,3.684529,4.489547,6.157468,2
5,eletronics,5.811656,3.572954,5.374311,1


In [4]:
data_knapsacks.head()

,max_weight,max_volume,max_categories
A,21,32,2
B,19,22,4
C,28,17,3


In [5]:
#Create sets of unique values of the sets
categories = data_items["category"].unique()
items_labels = data_items.index.unique().to_numpy()
knapsacks = data_knapsacks.index.unique().to_numpy()

## Create the model

In [6]:
model = pyo.ConcreteModel()

## Create the sets

- Set $C$, index $c$ - Categories
- Set $I$, index $i$ - Items
- Set $K$, index $k$ - Knapsacks

In [7]:
model.C = pyo.Set(initialize=categories)
model.I = pyo.Set(initialize=items_labels)
model.K = pyo.Set(initialize=knapsacks)

In [8]:
data_knapsacks.columns

Index(['max_weight', 'max_volume', 'max_categories'], dtype='object')

## Parameters

- $kw_k \space \forall \space k \in K$ - Knapsack weight capacity
- $kv_k \space \forall \space k \in K$ - Knapsack volume capacity
- $b_i \space \forall \space i \in I$ - Quantity available of each item
- $v_i \space \forall \space i \in I$ - Volume of each item
- $w_i \space \forall \space i \in I$ - Weight of each item
- $c_i \space \forall \space i \in I$ - Value of each item

In [9]:
knapsack_weights = data_knapsacks["max_weight"].to_dict()
knapsack_volumes = data_knapsacks["max_volume"].to_dict()

model.kw = pyo.Param(model.K, initialize=knapsack_weights)
model.kv = pyo.Param(model.K, initialize=knapsack_volumes)

In [10]:
items_volumes = data_items["volume"].to_dict()
items_weights = data_items["weight"].to_dict()
items_values = data_items["value"].to_dict()
items_available = data_items["available"].to_dict()

model.b = pyo.Param(model.I, initialize=items_available)
model.v = pyo.Param(model.I, initialize=items_volumes)
model.w = pyo.Param(model.I, initialize=items_weights)
model.c = pyo.Param(model.I, initialize=items_values)

## Create variables

- $x_{i, k}  \space \forall \space i \in I \space \forall \space k \in K$ - Number of items per knapsack

In [11]:
bounds_n_items = {}

for key, value in items_available.items():
    bounds_n_items[key] = (0, value)

def get_bounds(model, i, k):
    return bounds_n_items[i]

In [12]:
model.x = pyo.Var(model.I, model.K, within=pyo.Integers, bounds=get_bounds)

## Define constraints of number of items

$\displaystyle \sum_{i \in I} \sum_{k \in K} x_{i, k} \leq b_{i}$

In [13]:
def availability_constraint(model, i):
    
    return sum(model.x[i, k] for k in model.K) <= model.b[i]

model.availability_constraint = pyo.Constraint(model.I, rule=availability_constraint)

# Capacity constraints

$\displaystyle \sum_{i \in I} x_{i, k} v_{i} \leq kv_{k} \space \forall \space k \in K\\ \sum_{i \in I} x_{i, k} w_{i} \leq kw_{k} \space \forall \space k \in K$

In [14]:
def weight_constraint(model, k):
    
    return sum(model.x[i, k] * model.w[i] for i in model.I) <= model.kw[k]

model.weight_constraint = pyo.Constraint(model.K, rule=weight_constraint)

def volume_constraint(model, k):
    
    return sum(model.x[i, k] * model.v[i] for i in model.I) <= model.kv[k]

model.volume_constraint = pyo.Constraint(model.K, rule=volume_constraint)

## Objective function

$\displaystyle maximize \space \sum_{i \in I} \sum_{k \in K} x_{i, k} c_{i}$

In [15]:
def obj_function(model):
    
    return - sum(model.x[i, k] * model.c[i]\
        for i in model.I for k in model.K)
    
model.objective = pyo.Objective(rule=obj_function, sense=pyo.minimize)

# Optimize

In [16]:
model.write('multi_knapsacks_type1.lp', io_options={'symbolic_solver_labels': True})

('multi_knapsacks_type1.lp', 2279419354992)

In [17]:
opt = pyo.SolverFactory('glpk', executable='C:\\glpk-4.65\\w64\\glpsol.exe')
opt.options['tmlim'] = 120

In [18]:
solution_type1 = opt.solve(model)
print(solution_type1)


Problem: 
- Name: unknown
  Lower bound: -127.738437990416
  Upper bound: -127.738437990416
  Number of objectives: 1
  Number of constraints: 57
  Number of variables: 151
  Number of nonzeros: 451
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 32975
      Number of created subproblems: 32975
  Error rc: 0
  Time: 6.8684515953063965
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [19]:
model.objective.display()

objective : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : -127.738437990416


In [20]:
model.weight_constraint.display()

weight_constraint : Size=3
    Key : Lower : Body               : Upper
      A :  None : 20.983448820880767 :  21.0
      B :  None : 18.982435077352662 :  19.0
      C :  None :  27.99791623266966 :  28.0


In [21]:
model.volume_constraint.display()

volume_constraint : Size=3
    Key : Lower : Body               : Upper
      A :  None : 31.390966375697978 :  32.0
      B :  None : 21.194491520099053 :  22.0
      C :  None : 16.767874386429273 :  17.0


In [22]:
output_type1 = pd.DataFrame(index=items_labels, columns=knapsacks)

for i in items_labels:
    for k in knapsacks:
        output_type1.loc[i, k] = model.x[i, k].value

output_type1.to_excel("..\data\output_multiple_knapsacks_type1.xlsx")

## New constraint

The idea of this constraint is that each knapsack must contain at most items of some new distinct categories. Therefore, we must create new parameters, decision variables, and a new rule.

- $kc_k \space \forall \space k \in K$ - Knapsack maximum distinct categories
- $y_{c, k} \space \forall \space i \in I$ - Binary: is category on knapsack

$\displaystyle \sum_{c \in C} y_{c, k} \leq kc_{k} \space \forall \space k \in K$

In [23]:
max_categories_on_knapsack = data_knapsacks["max_categories"].to_dict()

#New parameter max categories on each knapsack
model.kc = pyo.Param(model.K, initialize=max_categories_on_knapsack)

#New variable binary if category is on knapsack
model.y = pyo.Var(model.C, model.K, initialize=0, within=pyo.Binary)

#Sum binary variables across categories for each knapsack
def constr_max_categories_on_k(model, k):
    return sum(model.y[c, k] for c in model.C) <= model.kc[k]

model.constr_max_categories_on_k = pyo.Constraint(model.K, rule=constr_max_categories_on_k)

### Weak formulation

$\displaystyle \sum_{i \in \{I \mid cat_i = c\}} x_{i, k} \leq y_{c, k} \space \forall \space k \in K \space \forall \space c \in C$

In [24]:
#Strategy 1 - Weak formulation
category_of_item = data_items["category"].to_dict()

def items_per_category_on_knapsack(model, c, k):
    return sum(model.x[i, k] if category_of_item[i] == c else 0
               for i in model.I)

model.x_cats_knapsacks = pyo.Expression(model.C, model.K, rule=items_per_category_on_knapsack)

#Suggestion: define big M
M = data_items.groupby("category")["available"].sum().to_dict()

model.M = pyo.Param(model.C, initialize=M)

#Constraint in which the binary is 1 is any item of category is in the knapsack
def constr_binary_cat_k(model, c, k):
    return model.x_cats_knapsacks[c, k] \
        <= model.y[c, k] * model.M[c]

model.constr_bin_cat_k = pyo.Constraint(model.C, model.K, rule=constr_binary_cat_k)

### Strong formulation

$\displaystyle x_{i, k} \leq y_{cat_i, k} \space \forall \space k \in K \space \forall \space i \in I$

In [25]:
#Stonger form of constraint in which the binary is 1 is any item of category is in the knapsack
def constr_binary_cat_item_k(model, i, k):
    return model.x[i, k] \
        <= model.y[data_items.loc[i, "category"], k] * model.b[i]

model.constr_binary_cat_item_k = pyo.Constraint(model.I, model.K, rule=constr_binary_cat_item_k)

In [26]:
model.write('multi_knapsacks_type2.lp', io_options={'symbolic_solver_labels': True})

('multi_knapsacks_type2.lp', 2279261836048)

In [27]:
solution_type2 = opt.solve(model)
print(solution_type2)


Problem: 
- Name: unknown
  Lower bound: -124.512403855581
  Upper bound: -124.512403855581
  Number of objectives: 1
  Number of constraints: 225
  Number of variables: 166
  Number of nonzeros: 931
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 11379
      Number of created subproblems: 11379
  Error rc: 0
  Time: 5.824418544769287
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [28]:
model.objective.display()

objective : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : -124.51240385558054


In [29]:
output_type2 = pd.DataFrame(index=items_labels, columns=knapsacks)

for i in items_labels:
    for k in knapsacks:
        output_type2.loc[i, k] = model.x[i, k].value

output_type2.to_excel("..\data\output_multiple_knapsacks_type2.xlsx")

In [30]:
model.constr_max_categories_on_k.display()

constr_max_categories_on_k : Size=3
    Key : Lower : Body : Upper
      A :  None :  2.0 :   2.0
      B :  None :  4.0 :   4.0
      C :  None :  3.0 :   3.0


In [31]:
model.weight_constraint.display()

weight_constraint : Size=3
    Key : Lower : Body               : Upper
      A :  None :  20.19892493022179 :  21.0
      B :  None :  18.86478343484082 :  19.0
      C :  None : 27.786104447512663 :  28.0


In [32]:
model.volume_constraint.display()

volume_constraint : Size=3
    Key : Lower : Body               : Upper
      A :  None :  31.15237230363154 :  32.0
      B :  None :  20.67223464031445 :  22.0
      C :  None : 16.215114639890935 :  17.0
